## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
import os

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tarakan,ID,2022-01-31 21:06:45,3.3000,117.6333,79.56,79,93,9.53,overcast clouds
1,1,Bourail,NC,2022-01-31 21:06:45,-21.5667,165.4833,78.73,78,100,3.00,overcast clouds
2,2,Hambantota,LK,2022-01-31 21:06:45,6.1241,81.1185,75.76,81,4,15.97,clear sky
3,3,Poum,NC,2022-01-31 21:06:46,-20.2333,164.0167,80.33,76,100,14.63,overcast clouds
4,4,Tamworth,GB,2022-01-31 21:06:46,52.6340,-1.6959,44.35,88,80,5.01,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tarakan,ID,2022-01-31 21:06:45,3.3000,117.6333,79.56,79,93,9.53,overcast clouds
1,1,Bourail,NC,2022-01-31 21:06:45,-21.5667,165.4833,78.73,78,100,3.00,overcast clouds
2,2,Hambantota,LK,2022-01-31 21:06:45,6.1241,81.1185,75.76,81,4,15.97,clear sky
3,3,Poum,NC,2022-01-31 21:06:46,-20.2333,164.0167,80.33,76,100,14.63,overcast clouds
6,6,Tautira,PF,2022-01-31 21:06:47,-17.7333,-149.1500,84.00,78,35,19.22,light rain
9,9,Arraial Do Cabo,BR,2022-01-31 21:06:49,-22.9661,-42.0278,80.46,78,75,3.44,broken clouds
12,12,Jutai,BR,2022-01-31 21:02:48,-5.1833,-68.9000,84.83,65,100,4.34,overcast clouds
14,14,Cape Town,ZA,2022-01-31 21:06:50,-33.9258,18.4232,76.12,78,7,5.75,clear sky
16,16,Kahului,US,2022-01-31 21:04:15,20.8947,-156.4700,79.97,64,20,8.05,few clouds
17,17,Atuona,PF,2022-01-31 21:06:51,-9.8000,-139.0333,78.48,76,19,21.59,light rain


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                195
City                   195
Country                195
Date                   195
Lat                    195
Lng                    195
Max Temp               195
Humidity               195
Cloudiness             195
Wind Speed             195
Current Description    195
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#Not Applicable

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tarakan,ID,79.56,overcast clouds,3.3000,117.6333,
1,Bourail,NC,78.73,overcast clouds,-21.5667,165.4833,
2,Hambantota,LK,75.76,clear sky,6.1241,81.1185,
3,Poum,NC,80.33,overcast clouds,-20.2333,164.0167,
6,Tautira,PF,84.00,light rain,-17.7333,-149.1500,
9,Arraial Do Cabo,BR,80.46,broken clouds,-22.9661,-42.0278,
12,Jutai,BR,84.83,overcast clouds,-5.1833,-68.9000,
14,Cape Town,ZA,76.12,clear sky,-33.9258,18.4232,
16,Kahului,US,79.97,few clouds,20.8947,-156.4700,
17,Atuona,PF,78.48,light rain,-9.8000,-139.0333,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)

In [12]:
#drop rows that have NaN values
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tarakan,ID,79.56,overcast clouds,3.3000,117.6333,RedDoorz Syariah near Taman Berlabuh Tarakan
1,Bourail,NC,78.73,overcast clouds,-21.5667,165.4833,Motel Allamanda
2,Hambantota,LK,75.76,clear sky,6.1241,81.1185,Bungalow 63
3,Poum,NC,80.33,overcast clouds,-20.2333,164.0167,Gîte kajeon
6,Tautira,PF,84.00,light rain,-17.7333,-149.1500,Pension Oaoa
...,...,...,...,...,...,...,...
651,Grand-Santi,GF,76.64,overcast clouds,4.2500,-54.3833,Hotel La Villa Couacou
658,Gamboma,CG,81.36,broken clouds,-1.8764,15.8644,Residence N'Keni
661,Tual,ID,83.84,moderate rain,-5.6667,132.7500,Grand Vilia Hotel
665,Tevaitoa,PF,81.01,clear sky,-16.7833,-151.5000,Hôtel Raiatea Lodge


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))